# Lending Club

<img src= https://creditkarmacdn-a.akamaihd.net/res/content/reviews/lending-club-personal-loans/small.png>

#### Autores: 
* Sergio Cañón Laíz
* Ignacio Ruiz de Zuazu Echevarría

#### Universidad de CUNEF
#### Máster en Data Science para Finanzas
#### 4 de diciembre de 2020

## Tabla de contenidos

* [1. Objetivos del trabajo](#Objetivos-del-trabajo) 
* [2. Dataset y variables](#Dataset-y-variables) 
* [3. Metodología](#Metodología)
* [4. Análisis Exploratorio](#Análisis-Exploratorio-de-los-Datos)
   * [4.1 Variable Target](#Variable-Target)
   * [4.2 Propósito del crédito](#Propósito-del-crédito)
   * [4.3 Distribuciones de la cantidad del préstamo y tipo de interés](#Distribuciones-de-la-cantidad-del-préstamo-y-tipo-de-interés)
   * [4.4 Estado de verificación](#Estado-de-verificación)
   * [4.5 Columna de pagos](#Columna-de-pagos)
   * [4.6 Pagos según el Loan Status](#Pagos-según-el-Loan-Status)
   * [4.7 Grades](#Grades)
   * [4.8 Terms](#Terms) 
* [5. Data engineering y selección de variables](#Data-engineering-y-selección-de-variables)
   * [5.1 Variables con NA](#Variables-con-NA)
   * [5.2 Tratamiento de variables categóricas](#Tratamiento-de-variables-categóricas)
   * [5.3 Transformación de dummies](#Transformación-de-dummies)
   * [5.4 Transformación de la variable Target](#Transformación-de-la-variable-Target)
   * [5.5 Dataset final para realizar modelos](#Dataset-final-para-realizar-modelos)




## Objetivos del trabajo

El objetivo de este proyecto es el de construir un modelo de aprendizaje automático para predecir la probabilidad de que un préstamo se cancele a partir de diversos modelos. Para este trabajo se emplearán los datos proporcionados por Lending Club, en la que se nos aporta información sobre el prestatario y los préstamos concedidos. 

### Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
from scipy import stats

# Para graficas en jupyter
%matplotlib inline
# Tamaño de las figuras
rcParams['figure.figsize'] = 14,6

## Dataset y variables

In [ ]:
loan_training_set1 = pd.read_csv('./data/01_raw/Loan_training_set_1_4.csv', sep = ',' , skiprows=1)
loan_training_set1.drop(loan_training_set1.index[105451], inplace=True)
loan_training_set1.drop(loan_training_set1.index[105451], inplace=True)
loan_training_set1

In [ ]:
loan_training_set2 = pd.read_csv('./data/01_raw/Loan_training_set_2_4.csv', sep = ',' , skiprows=1)
loan_training_set2.drop(loan_training_set2.index[122701], inplace=True)
loan_training_set2.drop(loan_training_set2.index[122701], inplace=True)
loan_training_set2

In [ ]:
loan_training_set3 = pd.read_csv('./data/01_raw/Loan_training_set_3_4.csv', sep = ',' , skiprows=1)
loan_training_set3.drop(loan_training_set3.index[235629], inplace=True)
loan_training_set3.drop(loan_training_set3.index[235629], inplace=True)
loan_training_set3

In [ ]:
loan_training_set4 = pd.read_csv('./data/01_raw/Loan_training_set_4_4.csv', sep = ',' , skiprows=1)
loan_training_set4.drop(loan_training_set4.index[421095], inplace=True)
loan_training_set4.drop(loan_training_set4.index[421095], inplace=True)
loan_training_set4

In [ ]:
frames = [loan_training_set1, loan_training_set2, loan_training_set3,loan_training_set4]
lendingclub = pd.concat(frames)

In [ ]:
lendingclub.info()

El dataset tiene 884.884 filas/observaciones y 151 columnas/variables. De las variables, 111 son de tipo numérico y 40 de tipo objeto.

Resumen de las variables numéricas

In [ ]:
lendingclub.describe()

## Metodología

## Análisis Exploratorio de los Datos

### Variable Target

La variable objetivo en este trabajo es `Loan Status`. Vamos a estudiar todos los status.

In [ ]:
total = len(lendingclub)
plt.figure(figsize = (14,13))
g = sns.countplot(x="loan_status", data=lendingclub, 
                  color='blue')
g.set_xticklabels(g.get_xticklabels(),rotation=45)
g.set_xlabel("Categorias de Loan Status", fontsize=12)
g.set_ylabel("Count", fontsize=15)
g.set_title("Distribución de tipos de Loan Status", fontsize=20)
sizes=[]
for p in g.patches:
    height = p.get_height()
    sizes.append(height)
    g.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}%'.format(height/total*100),
            ha="center", fontsize=12) 
g.set_ylim(0, max(sizes) * 1.10)
plt.show()

### Propósito del crédito

Propósito del crédito es una categoría proporcionada por el prestatario para la solicitud de préstamo. Como es una característica categórica que dice cuál es el propósito del préstamo, sería interesante comenzar por esta variable.

In [ ]:
plt.figure(figsize=(14,6))

g = sns.countplot(x='purpose', data=lendingclub, 
                  color='blue')
g.set_title("Propósito de Crédito de los Clientes", fontsize=22)
g.set_xlabel("Propósito", fontsize=18)
g.set_ylabel('Count', fontsize=18)

sizes=[]

for p in g.patches:
    height = p.get_height()
    sizes.append(height)
    g.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}%'.format(height/total*100),
            ha="center", fontsize=14) 
    
g.set_ylim(0, max(sizes) * 1.10)
g.set_xticklabels(g.get_xticklabels(),
                  rotation=45)

plt.show()

### Distribuciones de la cantidad del préstamo y tipo de interés

El monto indicado del préstamo solicitado por el prestatario. Si en algún momento, el departamento de crédito reduce el monto del préstamo, entonces se reflejará en este valor. `intRate` es la tasa de interés del préstamo.


In [ ]:
int_numeric = lendigclub["int_rate"].str.strip("%")
int_numeric = pd.to_numeric(int_numeric)
lendigclub["int_rate"] = int_numeric/100

lendigclub['int_round'] = lendigclub['int_rate'].round(0).astype(int)

#I will start looking the loan_amnt column
plt.figure(figsize=(14,10))

# Loan Amt plot
plt.subplot(211)
g = sns.distplot(lendigclub["loan_amnt"])
g.set_xlabel("Loan Amount Value", fontsize=16)
g.set_ylabel("Frequency", fontsize=16)
g.set_title("Loan Amount Distribuition", fontsize=20)

## Interest plot
plt.subplot(212)
g1 = sns.countplot(x="int_round", data=lendigclub, 
                  color='blue')
g1.set_xlabel("Loan Interest Rate", fontsize=16)
g1.set_ylabel("Count", fontsize=16)
g1.set_title("Interest Rate Distribuition", fontsize=20)
sizes=[] # Get highest values in y
for p in g1.patches:
    height = p.get_height()
    sizes.append(height)
    g1.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}%'.format(height/total*100),
            ha="center", fontsize=12) 
g1.set_ylim(0, max(sizes) * 1.10) # set y limit based on highest heights

plt.subplots_adjust(hspace = 0.4,top = 0.9)

plt.show()


### Estado de verificación

Indica si el ingreso conjunto de los coprestatarios fue verificado por Lending Club, no verificado, o si se verificó la fuente de ingresos.

In [ ]:
plt.figure(figsize = (13,6))

g = sns.countplot(x="verification_status", data=lendingclub, 
                  color='blue')
g.set_xlabel("Categorias de Loan Status", fontsize=15)
g.set_ylabel("Count", fontsize=15)
g.set_title("Distribucion tipo de Loan Status", fontsize=20)
sizes=[]
for p in g.patches:
    height = p.get_height()
    sizes.append(height)
    g.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}%'.format(height/total*100),
            ha="center", fontsize=14) 
g.set_ylim(0, max(sizes) * 1.10)

plt.show()

### Columna de pagos

El pago mensual adeudado por el prestatario si se origina el préstamo. Con esta información, podemos investigar la diferencia entre `emp_title` o regiones, para encontrar algunos patrones de valores interesantes para el estudio.

In [ ]:
plt.figure(figsize=(12,5))

sns.distplot(lendingclub['installment'])
plt.title("Installment Distribution", fontsize=20)
plt.xlabel("Installment Range", fontsize=17)
plt.ylabel("Density", fontsize=17)

plt.show()

### Pagos según el Loan Status

In [ ]:
plt.figure(figsize = (14,12))

plt.subplot(211)
g = sns.violinplot(x='loan_status', y="installment",
                   data=lendingclub, color='blue')
g.set_xticklabels(g.get_xticklabels(),rotation=45)
g.set_xlabel("Loan Status", fontsize=17)
g.set_ylabel("Pago", fontsize=17)
g.set_title("Cuotas por estado del préstamo", fontsize=20)

plt.subplot(212)
g1 = sns.violinplot(x='loan_status', y="total_acc",
                   data=lendingclub, color='blue')
g1.set_xticklabels(g.get_xticklabels(),rotation=45)
g1.set_xlabel("Loan Status", fontsize=17)
g1.set_ylabel("Líneas de cuenta totales", fontsize=17)
g1.set_title("Distribución total de líneas de cuenta por estado de préstamo", fontsize=20)

plt.subplots_adjust(hspace = 0.5,top = 0.9)

plt.show()

### Grades

Una de las variables que encontramos relevantes para realizar los modelos es `Grades`, por ello vamos a estudiarla.

In [ ]:
order_sub = lendingclub.groupby("sub_grade")['int_rate'].count().index

plt.figure(figsize=(14,16))

plt.suptitle('Grade and Sub-Grade Distributions \n# Interest Rate and Loan Amount #', fontsize=22)

plt.subplot(311)
g = sns.boxplot(x="grade", y="loan_amnt", data=lendingclub,
                palette="hls", hue="application_type", 
                order=["A",'B','C','D','E','F', 'G'])
g.set_xlabel("Grade Values", fontsize=17)
g.set_ylabel("Loan Amount", fontsize=17)
g.set_title("Lending Club Loan - Loan Amount Distribution by Grade", fontsize=20)
g.legend(loc='upper right')

plt.subplot(312)
g1 = sns.boxplot(x='grade', y="int_rate",data=lendingclub, 
               hue="application_type", palette = "hls",  
               order=["A",'B','C','D','E','F', 'G'])
g1.set_xlabel("Grade Values", fontsize=17)
g1.set_ylabel("Interest Rate", fontsize=17)
g1.set_title("Lending Club Loan - Interest Rate Distribution by Grade", fontsize=20)

plt.subplot(313)
g2 = sns.boxenplot(x="sub_grade", y="int_rate", data=lendingclub, 
                   palette="hls", order=order_sub)
g2.set_xlabel("Sub Grade Values", fontsize=15)
g2.set_ylabel("Interest Rate", fontsize=15)
g2.set_title("Lending Club Loan - Interest Rate Distribution by Sub-Grade", fontsize=20)

plt.subplots_adjust(hspace = 0.4,top = 0.9)

plt.show()

### Terms

In [ ]:
order_sub = lendingclub.groupby("sub_grade")['int_rate'].count().index

plt.figure(figsize=(14,18))

plt.suptitle('Term Distributions \n# Count, Interest Rate and Loan Amount #', fontsize=22)

plt.subplot(311)
g = sns.countplot(x="term", data=lendingclub,color='blue')
g.set_xlabel("Term Values", fontsize=17)
g.set_ylabel("Count", fontsize=17)
g.set_title("Lending Club Loan \nTerm Count Distribution", fontsize=20)
sizes=[]
for p in g.patches:
    height = p.get_height()
    sizes.append(height)
    g.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}%'.format(height/total*100),
            ha="center", fontsize=14) 
g.set_ylim(0, max(sizes) * 1.10)

plt.subplot(312)
g1 = sns.violinplot(x='term', y="int_rate",data=lendingclub )
g1.set_xlabel("Term Values", fontsize=17)
g1.set_ylabel("Interest Rate", fontsize=17)
g1.set_title("Lending Club Loan \nInterest Rate Distribution by Term Values", fontsize=20)

plt.subplot(313)
g2 = sns.violinplot(x="term", y="loan_amnt", data=lendingclub)
g2.set_xlabel("Term Values", fontsize=17)
g2.set_ylabel("Loan Amount", fontsize=17)
g2.set_title("Lending Club Loan \nLoan Amount Distribution by Term Values", fontsize=20)

plt.subplots_adjust(hspace = 0.4, top = 0.9)

plt.show()

## Data engineering y selección de variables

### Variables con NA

En primer lugar, quitamos todos aquellas variables con un número considerable de valores perdidos que pueden influir negativamente en nuestros modelos. En este caso consideramos quitar todas aquellas variables con más de 431,425 NA. Podemos observar el número de missing values en la siguiente tabla.

In [ ]:
na_columnas = pd.DataFrame(lendingclub.isna().sum(), columns = ['Valores NaN']).sort_values(['Valores NaN'], ascending = False)
pd.set_option('display.max_rows', nacolumnas.shape[0]+1)
na_columnas

In [ ]:
lendingclub1 = lendingclub.dropna(thresh=len(lendingclub) - 431425, axis=1)

Comporobamos

In [ ]:
na_columnas1  = pd.DataFrame(lendingclub1.isna().sum(), columns = ['Valores NaN']).sort_values(['Valores NaN'], ascending = False)
pd.set_option('display.max_rows', na_columnas1.shape[0]+1)
na_columnas1

Guardamos este dataset por si nos hiciese falta en un futuro del trabajo

In [ ]:
lendingclub1.to_csv('./data/02_intermidiate_general.csv', index = False)

A continuación, vamos a observar las variables de tipo object y ver cuantos tipos de valores contienen.

In [ ]:
lendingclub1.select_dtypes("object").nunique().sort_values()

Observamos que hay muchos tipos de valores en `url`, lo que podría ser un impedimento a la hora de realizar nuestro modelos así como escasa aportación de información a la hora de determinar el `loan_status`.

Y vemos los NA que contienen

In [ ]:
lendingclub1.select_dtypes("object").isna().sum()

A continuación, mirando la información que nos puede aportar cada variable a la hora de ver si otorgamos un préstamo o no a un cliente, hemos considerado una serie de variables que no resultan relevantes para el objetivo de este trabajo.

In [ ]:
lendigclub_v2 = lendingclub1.drop(['id',
 'funded_amnt',
 'funded_amnt_inv',
 'installment',
 'emp_title',
 'emp_length',
 'issue_d',
  "zip_code",                     
 'pymnt_plan',
 'url',
 'title',
 'addr_state',
 'dti',
 'earliest_cr_line',
 'fico_range_low',
 'fico_range_high',
 'inq_last_6mths',
 'pub_rec',
 'initial_list_status',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_late_fee',                   
 'last_pymnt_d',
 'last_pymnt_amnt',
 'last_credit_pull_d',
 'last_fico_range_high',
 'last_fico_range_low',
 'policy_code',
 'application_type',
 'bc_open_to_buy',
 'mo_sin_old_il_acct',
 'mo_sin_rcnt_rev_tl_op',
 'mo_sin_rcnt_tl',
 'mths_since_recent_bc',
 'num_actv_bc_tl',
 'num_bc_sats',                   
 'num_rev_accts',
 'num_tl_120dpd_2m',
 'num_tl_90g_dpd_24m',
 'num_tl_op_past_12m',
 'pct_tl_nvr_dlq',
 'disbursement_method',
 'debt_settlement_flag',
  "mths_since_recent_inq",
"bc_util",
 "percent_bc_gt_75",
 "revol_util",
"avg_cur_bal" ], axis = 1)

### Tratamiento de variables categóricas

Las varaibles de `term` y `int_rate` Contienen variables categóricas que habría que pasar a numéricas

In [ ]:
sin_months = lendigclub_v2["term"].str.strip("months")
sin_months = pd.to_numeric(sin_months)
lendigclub_v2["term"] = sin_months

In [ ]:
int_numeric = lendigclub_v2["int_rate"].str.strip("%")
int_numeric = pd.to_numeric(int_numeric)
lendigclub_v2["int_rate"] = int_numeric/100

Comprobamos que se han cambiado de tipo

In [ ]:
lendigclub_v2.info()

### Transformación de dummies

In [ ]:
X = lendigclub_v2.drop(['loan_status'], axis=1)
encoded_all = pd.get_dummies(X)
df_c = pd.concat([encoded_all, lendigclub_v2['loan_status']], axis=1)
df_c_prueba_sin_current = df_c[df_c["loan_status"]!="Current"]

### Transformación de la variable Target

Vemos que en nuestra variable target tenemos 7 valores posibles, para este trabajo haremos un encode entre las que sean Fully Paid y el resto de variables. Por lo tanto las que sean Fully Paid tendrán como valor el 1 y el resto tomarán el valor 0.

En resumen, vamos a diferenciar entre los clientes que pagan y no pagan. Diferenciamos entre:

* Pagan: 'Fully Paid'.

* No pagan (se retrasan aunque sea un mínimo): 'Late (31-120 days)','Late (16-30 days)', 'Charged Off', 'Default'.

In [ ]:
lendingclub1["loan_status"].value_counts()

In [ ]:
df_c_prueba_sin_current['loan_status'].mask(df_c_prueba_sin_current['loan_status'] == 'Fully Paid', 1, inplace=True)
df_c_prueba_sin_current['loan_status'].mask(df_c_prueba_sin_current['loan_status'] != 1, 0, inplace=True)


### Dataset final para realizar modelos

In [ ]:
df_c_prueba_sin_current.to_csv('./data/03_processed.csv', index = False)